In [777]:
# Python Code for scrapping Hospital data from 'http://sais.health.pa.gov/commonpoc/dohqalocatorcommon.asp'
# Using BeautifulSoup Library
from bs4 import BeautifulSoup
import urllib

# Copy the required county URL from the above link
r = urllib.urlopen('http://sais.health.pa.gov/commonpoc/content/publiccommonpoc/commonpocselect.asp?FORMSUBMITTED=normalSearch&selcty=Allegheny').read()
soup = BeautifulSoup(r)
import pandas as pd
import re

In [778]:
# Getting Hospital data in from table 1 of soup object
table = soup.find_all('table')[1]
# Getting county name from header in table 0
countyinfo=soup.find_all('table')[0]
countyname=str(countyinfo.find_all('strong')[6].next.next.strip())


In [779]:
#Getting Rows 
rows = table.find_all('tr')


In [780]:
#Creating a dictionary to store data
data = {
    'Name' : [],
    'Type_Facility' : [],
    'Status' : [],
    'Address' : [],
    'city_state_zip' : [],
    'Patient_Survey_Link' : [],
    'Building_Survey_Link' :[],
    'County':[]
}
# Creating prefix link for Patient Survey Link and Building survey Link
prefix="http://sais.health.pa.gov/commonpoc/content/publiccommonpoc/"

In [781]:
# For loop to append data in dictionary 
# find_all() to fetch the required value

for row in rows:
    data['Name'].append( row.find_all('b')[0].next )
    data['Type_Facility'].append( row.find_all('b')[1].next.next )
    data['Status'].append( row.find_all('b')[2].next.next )
    data['Address'].append( row.find_all('b')[2].next.next.next.next )
    data['city_state_zip'].append( row.find_all('b')[2].next.next.next.next.next.next )
    data['County'].append( countyname )
    link = row.find_all(href=True)
    
# Fetching links for Patient Suvery Data and Building Survey Data
# Check if link exist for Patient Survey data
    check=re.search('QAsurvey',str(row.find_all('a')))
    if check<>None:
        data['Patient_Survey_Link'].append( prefix+link[0]['href'] )
# If link is not present, append comments
    else:
        data['Patient_Survey_Link'].append( 'No Available Health Surveys' )
#Check if link exist for Building Survey data        
    check2=re.search('Building',str(row.find_all('strong')))
    
    if check2==None:
        data['Building_Survey_Link'].append( prefix+link[1]['href']) 
# If link is not present, append comments
    else:
        data['Building_Survey_Link'].append( 'Building Safety surveys are not required for this facility' )

In [782]:
# Creating a data frame data dictionary
HospitalData = pd.DataFrame( data )
#Exporting dataframe as csv with countyname prefixed
HospitalData.to_csv(countyname+"_HospitalData.csv",columns=["Name","Type_Facility","Status","Address","city_state_zip","County","Patient_Survey_Link","Building_Survey_Link"])